In [ ]:
# default_exp constraints
# hide
_FNAME='constraints'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted constraints.ipynb.


In [ ]:
#export
import logging
logger = logging.getLogger()
import traceback
from neo4j.exceptions import ClientError
from mranderson import Query

def uniqueness(label:str, property_name:str, safely=True):
    '''
    Adds a uniqueness constraint on the property property_name on nodes of label
    e.g. uniqueness('Device', 'device_id') --> Device nodes must have a unique 'device_id' property
    
    safely: If False, throw an exception if the constraint already exists
    '''
    constraint = "CREATE CONSTRAINT ON (node:{label}) ASSERT node.{property_name} IS UNIQUE".format(
            label=label, property_name=property_name)
    q = Query()
    q.add(constraint)
    try:
        resp = q.create()
    except ClientError as e:
        logger.info("Constraint {} already exists".format(constraint))
        if not safely:
            raise e

In [ ]:
from mranderson.sandbox import start_neo4j, start_fresh, create_totem
container_name = 'mranderson_constraints_unittest'
container, driver = start_neo4j(container_name=container_name)

In [ ]:
import mranderson.node as node
from neo4j.exceptions import ConstraintError

def test_uniqueness():
    TESTCASE.assertTrue(start_fresh())
    create_totem()
    
    uniqueness('Test', 'eid')
    #creating the constraint safely is idempotent
    uniqueness('Test', 'eid', safely=True)
    
    node.create("(:Test {eid:1})")
    
    #new nodes with eid=1 cannot be created
    with TESTCASE.assertRaises(ConstraintError):
        node.create("(:Test {eid:1})")

    #nodes with different eid are OK
    node.create("(:Test {eid:2})")
    
    #creating the constraint again without `safely` fails
    with TESTCASE.assertRaises(ClientError):
        uniqueness('Test', 'eid', safely=False)
    print("SUCCESS")
                
test_uniqueness()

SUCCESS


In [ ]:
container.stop()